# Microfluidics flow past Yeast Cell with SpatialPy

## Definition of the model

### Imports and definitions

In [1]:
import os
import sys
import math
sys.path.insert(1, "../../")
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [2]:
class Fluid(spatialpy.Geometry):
    def __init__(self, external_radius):
        self.ers = external_radius**2
        
    def create_particles(self, domain, xmin, xmax, ymin, ymax, zmin, zmax, delta):
        Np = 0
        for x in np.arange(xmin, xmax+delta, delta):
            for y in np.arange(ymin, ymax+delta, delta):
                for z in np.arange(zmin, zmax+delta, delta):
                    if self.inside((x, y, z), False):
                        domain.add_point([x, y, z], type=0, vol=1, mass=1, nu=0, fixed=False)
                        Np += 1
        return Np
        
    def inside(self, x, on_boundary):
        return x[0]**2 + x[1]**2 + x[2]**2 > self.ers

In [3]:
class CellWall(spatialpy.Geometry):
    def __init__(self, external_radius, internal_radius):
        self.ers = external_radius**2
        self.irs = internal_radius**2
        
    def create_particles(self, domain, external_radius, delta):
        Np = 0
        for x in np.arange(-external_radius, external_radius+delta, delta):
            for y in np.arange(-external_radius, external_radius+delta, delta):
                for z in np.arange(-external_radius, external_radius+delta, delta):
                    if self.inside((x, y, z), False):
                        domain.add_point([x, y, z], type=0, vol=1, mass=1, nu=0, fixed=False)
                        Np += 1
        return Np
        
    def inside(self, x, on_boundary):
        radius = x[0]**2 + x[1]**2 + x[2]**2
        return radius < self.ers and radius > self.irs

### Model

In [4]:
class MicrofluidicsFlow(spatialpy.Model):
    FLUID = 1
    
    def __init__(self, model_name="Microfluidics Flow past Yeast Cell"):
        spatialpy.Model.__init__(self, model_name)
        
        # System Constants
        Lxint = 80e-6
        Lyint = 40e-6
        Lzint = 20e-6
        radius_external = 2.5e-6
        radius_internal = 2.4e-6
        rhof = rhoc = 1000
        
        # Species
        
        # Discretization
        Npx = 40
        
        # Compute domain bounds (including the boundary)
        deltaf = Lxint/Npx
        deltac = deltaf/3
        xmin = -Lxint/2
        xmax = Lxint/2
        ymin = -Lyint/2
        ymax = Lyint/2
        zmin = -Lzint/2
        zmax = Lzint/2
        
        # Domain
        domain = spatialpy.Domain(
            0, xlim=(xmin, xmax), ylim=(ymin, ymax), zlim=(zmin, zmax), gravity=[0, -1, 0]
        )
        # Fluid Region
        fluid_region = Fluid(radius_external)
        Nfp = fluid_region.create_particles(domain, xmin, xmax, ymin, ymax, zmin, zmax, deltaf)
        # Cell Wall Region
        cell_region = CellWall(radius_external, radius_internal)
        Ncp = cell_region.create_particles(domain, radius_external, deltac)
        
        self.add_domain(domain)
        
        # Compute volume and mass per particle
        vol = Lxint*Lyint*Lzint
        vol_cell_int = 4/3*math.pi*math.pow(radius_internal, 3)
        vol_cell_ext = 4/3*math.pi*math.pow(radius_external, 3)
        vol_cell_wall = vol_cell_ext-vol_cell_int
        vol_fluid = vol-vol_cell_ext
        mfluid = vol_fluid*rhof/Nfp
        mcell = vol_cell_wall*rhoc/Ncp
        
        # Types
        
        # Static Domain
        
        # Initial Conditions
        
        # Boundary Conditions
        
        # Timespan
        dt = 1e-6
        nt = 10000000
        freq_results = 100
        self.timespan(np.arange(0, (nt + 1) * dt, freq_results * dt), timestep_size=dt)

In [ ]:
model = MicrofluidicsFlow()

In [ ]:
model.domain